In [1]:
import os
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "Web_Django.settings")

import django
django.setup()

In [2]:
from app01 import models
from django.db.models import Max, Min, Avg, Sum, Count, F, Q

In [5]:
# 聚合示例，可对 models.Book.objects 或 QuerySet 进行 aggregate
print(models.Book.objects.aggregate(Max('price')))
print(models.Book.objects.aggregate(Min('price')))
print(models.Book.objects.aggregate(Avg('price')))
print(models.Book.objects.aggregate(Count('price')))
print('\n')
print(models.Book.objects.all().aggregate(Max('price')))
print(models.Book.objects.all().aggregate(Min('price')))
print(models.Book.objects.all().aggregate(Avg('price')))
print(models.Book.objects.all().aggregate(Count('price')))

{'price__max': 150}
{'price__min': 10}
{'price__avg': 61.2222}
{'price__count': 9}


{'price__max': 150}
{'price__min': 10}
{'price__avg': 61.2222}
{'price__count': 9}


In [28]:
# 聚合与命名
print(models.Book.objects.filter(id__gt=5).aggregate(Max('price')))
print(models.Book.objects.filter(id__gt=5).aggregate(Avg('price')))
print(models.Book.objects.filter(id__gt=5).aggregate(Max('price'), Avg('price')))
print(models.Book.objects.filter(id__gt=5).aggregate(草泥马=Max('price'), b=Avg('price')))

{'price__max': 60}
{'price__avg': 35.1667}
{'price__max': 60, 'price__avg': 35.1667}
{'草泥马': 60, 'b': 35.1667}


In [55]:
# annotate 返回QuerySet，只是在源数据对象中多加了aggregate属性

# 获取每个出版社最便宜的书 
# 方法一：在Press表按Press annotate然后以跨表信息aggregate
ret = models.Press.objects.all().annotate(min=Min('book__price'))
for i in ret:
    print(i.min)

print('\n')

# 方法二：在Book表按Press annotate然后以本表信息aggregate
ret = models.Book.objects.values('press__name').annotate(min=Min('price'))
for i in ret:
    print(i)

80
35
10
23


{'press__name': '广东工业大学出版社', 'min': 35}
{'press__name': '沙雕出版社', 'min': 10}
{'press__name': '沙河出版社', 'min': 23}
{'press__name': '仿生实验室出版社', 'min': 80}


In [72]:
# F表内查询 - F本质就是动态取值
ret = models.Book.objects.filter(id__gt=F('price')).values()
for i in ret:
    print(i)
    
ret = models.Book.objects.filter(price__lt=F('id')).values()
for i in ret:
    print(i)
    
# F表内根据原数据批量变化
models.Book.objects.all().update(price=F('price')*2)


{'id': 11, 'book_id': 'd00002', 'name': '沙河粉村相册', 'press_id': 5, 'price': 8, 'pub_date': datetime.date(2019, 5, 7)}
{'id': 11, 'book_id': 'd00002', 'name': '沙河粉村相册', 'press_id': 5, 'price': 8, 'pub_date': datetime.date(2019, 5, 7)}


9

In [17]:
# Q关系查询 - 为查询添加或且非关系运算 优先级  ~ > & > |
print(Q(id__lt=4), type(Q(id__lt=4)))
print((Q(id__lt=4) | ~Q(id__gt=8) | Q(id__gt=0)) & Q(id__lt=4))
print(~Q(id__gt=8))

print('\n')

a = Q(id__lt=4) | Q(id__gt=8)
ret = models.Book.objects.filter(a)
for i in ret:
    print(i)

(AND: ('id__lt', 4)) <class 'django.db.models.query_utils.Q'>
(AND: (OR: ('id__lt', 4), (NOT (AND: ('id__gt', 8))), ('id__gt', 0)), ('id__lt', 4))
(NOT (AND: ('id__gt', 8)))


a00001-Benjamin传记
c00002-沙雕古富源
d00001-沙河粉的故事
d00002-沙河粉村相册


In [106]:
# transaction - 有异常则放弃所有执行（只针对于ORM操作，亲测print无效）
from django.db import transaction

try:
    with transaction.atomic():
        print(1)
        print(2)
        int('ac')
        print(3)
        print(4)
except Exception as e:
    print(e)

1
2
invalid literal for int() with base 10: 'ac'


In [110]:
try:
    with transaction.atomic():
        models.Press.objects.create(name='超级出版社3')
        int('ac')
        models.Press.objects.create(name='超级出版社4')
except Exception as e:
    print(e)

invalid literal for int() with base 10: 'ac'


In [8]:
# 查找每个出版社价格最高的书籍价格
# 1.从出版社找
ret = models.Press.objects.values('name').annotate(max_price=Max('book__price')).values('max_price')
for i in ret:
    print(i)
print('\n')

# 2.从书籍找
ret = models.Book.objects.values('press__name').annotate(max_price=Max('price')).values('max_price')
for i in ret:
    print(i)
print('\n')

# 查找每个出版社的名字以及出的书的数量
# 1. 从出版社找
ret = models.Press.objects.values('name').annotate(book_count=Count('book')).values('name', 'book_count')
for i in ret:
    print(i)
print('\n')

# 2. 从书籍找
ret = models.Book.objects.values('press__name').annotate(book_count=Count('id')).values('press__name', 'book_count')
for i in ret:
    print(i)
print('\n')


{'max_price': 150}
{'max_price': 55}
{'max_price': 28}
{'max_price': 23}
{'max_price': None}
{'max_price': None}


{'max_price': 150}
{'max_price': 55}
{'max_price': 28}
{'max_price': 23}


{'name': '广东工业大学出版社', 'book_count': 2}
{'name': '沙雕出版社', 'book_count': 2}
{'name': '沙河出版社', 'book_count': 2}
{'name': '仿生实验室出版社', 'book_count': 3}
{'name': '超级出版社1', 'book_count': 0}
{'name': '超级出版社2', 'book_count': 0}


{'press__name': '广东工业大学出版社', 'book_count': 2}
{'press__name': '沙雕出版社', 'book_count': 2}
{'press__name': '沙河出版社', 'book_count': 2}
{'press__name': '仿生实验室出版社', 'book_count': 3}




In [12]:
ret = models.Book.objects.filter(price__gt=models.Book.objects.aggregate(avg=Avg('price')).get('avg'))
for i in ret:
    print(i.name, i.price)
print(models.Book.objects.aggregate(avg=Avg('price')).get('avg'))



Benjamin传记 110
Benjamin与Stella的爱情故事 80
Bionic的崛起 150
55.0


In [15]:
models.Book.objects.filter(name__contains=F('press__name'))

<QuerySet [<Book: a00001-Benjamin传记>, <Book: a00002-Benjamin与Stella的爱情故事>]>